In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data Import and Prep ##

In [2]:
data = pd.read_csv('train.csv')

In [97]:
data = np.array(data)
np.random.shuffle(data)

# Split the data to train and test
train_data = data[:33600, 1:].transpose()
train_labels = data[:33600 ,0]

test_data = data[33601:, 1:].transpose()
test_labels = data[33601:, 0]

In [161]:
w1 = np.random.randn(10,784) * 0.01
b1 = np.random.randn(10, 1)

## Manual Testing ##

In [162]:
z1 = np.matmul(w1, train_data) + b1

In [123]:
z1.shape

(10, 33600)

In [124]:
def Relu(matrix):
    for i in range(0, matrix.shape[1]):
        for j in range(0, 10):
            if matrix[j,i] < 0:
                matrix[j,i] = 0
            
    return matrix

In [125]:
a1 = Relu(z1)

In [130]:
a1

array([[   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [1664.97589461,    0.        , 1023.51196007, ...,    0.        ,
         905.42105208,  349.23207221],
       [1764.16562412, 3270.88019555,    0.        , ..., 3533.6729674 ,
        1832.45166582,    0.        ],
       ...,
       [1186.29238091, 3514.72689112,    0.        , ...,    0.        ,
        1535.31065586, 3331.28267939],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [1294.27118489,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ]])

In [131]:
w2 = w1 = np.random.randn(10,10) * 0.01
b2 = np.random.zeros(10, 1)

In [132]:
z2 = np.matmul(w2, a1) + b2

In [133]:
def Softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Numerical stability improvement
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

In [134]:
a2 = Softmax(z2)

In [135]:
a2.shape

(10, 33600)

In [144]:
train_data.shape

(784, 33600)

## Code Rewritten in Class format (Locked In)

In [158]:
def Relu(matrix):
    for i in range(0, matrix.shape[1]):
        for j in range(0, 10):
            if matrix[j,i] < 0:
                matrix[j,i] = 0
            
    return matrix

def Softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Numerical stability improvement
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

class Neural_Network:
    def __init__(self):
        self.w1 = np.random.randn(10, 784) * 0.01
        self.b1 = np.random.randn(10, 1)   * 0.01
        self.w2 = np.random.randn(10, 10)  * 0.01
        self.b2 = np.random.randn(10, 1)   * 0.01
        
    def forward_pass(self, a0):
        # Hidden Layer
        z1 = np.matmul(w1, a0) + b1
        a1 = Relu(z1)
        
        # Output Layer
        z2 = np.matmul(w2, a1) + b2
        a2 = Softmax(z2)
        
        return a2
    
    def cost_function(self):
        pass
    
    def backward_pass(self):
        pass

In [159]:
x = Neural_Network()
x.forward_pass(train_data)

ValueError: operands could not be broadcast together with shapes (10,33600) (1,10) 

(784, 33600)